# Scrape Top 100 Beers in BC From BeerAdvocate Website

### Import libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import re

### Get Beer Links 

In [2]:
page = requests.get('https://www.beeradvocate.com/beer/top-rated/ca/bc/')
soup = BeautifulSoup(page.content, 'html.parser')

beer_urls = []
for a in soup.find_all('a', href=True): 
    find_link = re.findall(r"^/beer/profile/[0-9]+/[0-9]+/$",a['href'])
    if find_link != []:
        beer_urls.append(find_link)

beer_links = ['https://www.beeradvocate.com' + s[0] for s in beer_urls]
beer_links[0:5]

['https://www.beeradvocate.com/beer/profile/32184/124777/',
 'https://www.beeradvocate.com/beer/profile/18858/53309/',
 'https://www.beeradvocate.com/beer/profile/18858/62911/',
 'https://www.beeradvocate.com/beer/profile/10321/49058/',
 'https://www.beeradvocate.com/beer/profile/18858/65392/']

### Loop The Link 

In [3]:
def beer_information(beer_links):
    beer_name = []
    beer_style = []
    beer_company = []
    beer_location_in_BC = []
    beer_abv = []
    beer_rating = []
    beer_description = []

    for link in beer_links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')

        # general information
        info1 = soup.find("meta",{"name":"keywords"}).get("content")
        info1 = info1.split(",") 

        beer_name.append(info1[0])
        beer_style.append(info1[1])
        beer_company.append(info1[2])
        beer_location_in_BC.append(info1[3])

        # Stats
        info2 = soup.find_all("dd", {"class":"beerstats"})

        stats = []
        for i in info2:
            find_b = i.find('b')
            if find_b != None:
                stats.append(find_b.text)

        beer_abv.append(stats[1])
        beer_rating.append(stats[-1])
        
        # BEER DESCRIPTION
        global notes
        info3 = [i.text for i in soup.select("div")]
        for i in info3:
            find_notes = re.findall("(Notes:)\n.+",i)
            if find_notes != []:
                notes = re.sub('Notes:|\n|\t','',i)
        beer_description.append(notes)
    
    return beer_name, beer_style, beer_company, beer_location_in_BC, beer_abv, beer_rating, beer_description

In [4]:
beer_name, beer_style, beer_company, beer_location_in_BC, beer_abv, beer_rating,beer_description = beer_information(beer_links)

In [5]:
beer_info_final = pd.DataFrame({'Name': beer_name,
                               'Style': beer_style,
                               'Manufacturer': beer_company,
                               'Location': beer_location_in_BC,
                               'ABV': beer_abv,
                               'Ratings': beer_rating,
                               'Description': beer_description})
beer_info_final

,Name,Style,Manufacturer,Location,ABV,Ratings,Description
0,Nectarous,Wild Ale,Four Winds Brewing Co.,Delta,5.5%,4.45,"In the pursuit of sour, peach and nectarine pl..."
1,Sartori Harvest IPA,IPA - American,Driftwood Brewery,Victoria,7%,4.4,This wet-hopped IPA can only be brewed once a ...
2,Fat Tug IPA,IPA - American,Driftwood Brewery,Victoria,7%,4.27,Fat Tug is a northwest style India Pale Ale th...
3,Red Racer IPA (India Pale Ale),IPA - American,Central City Brewers + Distillers,Surrey,6.5%,4.25,A style of ale that was fashioned to survive t...
4,Singularity,Stout - Russian Imperial,Driftwood Brewery,Victoria,10.6%,4.28,"2011-11.8%(silver foil), 2012-11.6%(black wax)..."
...,...,...,...,...,...,...,...
95,Pretty Much Yeah,IPA - American,Superflux,Vancouver,6.5%,4.18,A strong hoppy ale. Brewed specifically for th...
96,Disruption,IPA - Black / Cascadian Dark Ale,Category 12 Brewing,Saanichton,6.7%,3.97,Compulsion will drive you to distraction. Whil...
97,GIBitter,Bitter - English,Granville Island Brewery,Vancouver,4.8%,4.11,This single hop session bitter has big flavour...
98,Hop Head Double India Pale Ale,IPA - Imperial,Tree Brewing,Kelowna,8.3%,3.92,"At Tree Brewing Company, we take craft brewing..."


In [6]:
# beer_info_final.to_csv('beerinfo.csv', index=False)